In [1]:
import pandas as pd
csv='data/2019-04_2019-08_GyokuSendo.csv'

In [2]:
df = pd.read_csv(csv,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])
df.rename(columns = {'serial':'ap_id','tlm_datagettime':'timestamp'}, inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df=df.sort_values(by=['timestamp'])

In [3]:
#df_710 =df.query("ap_id=='AP710906'")
#df_710.head()

,ap_id,tripid,tripcount,timestamp,lon,lat


In [4]:
arr_ap_ids = df.ap_id.unique()
#ap_id = arr_ap_ids[0]
#ap_id

arr_trip_summary = []

for ap_id in arr_ap_ids:
    df_single_ap = df.query("ap_id=='"+ap_id+"'").copy()
    df_single_ap = df_single_ap.sort_values(by=['timestamp'])
    arr_trips = df_single_ap.tripcount.unique()
    #trip_count = arr_trips[0]
    #trip_count
    ts_prevs = df_single_ap.timestamp.min()
    lat_prevs = df_single_ap.iloc[0].lat
    lon_prevs = df_single_ap.iloc[0].lon
    
    trip_count_prevs = -9
    for trip_count in arr_trips:
        
        
        if (trip_count -1) != trip_count_prevs: # do not process trips which are not consicutive  e.g. trip_count_prevs = 10 and  trip_count= 13
            trip_count_prevs = trip_count
            continue
         
        # process consicutive trips only e.g. trip_count_prevs = 12 and  trip_count=13
        df_single_trip = df_single_ap.query("tripcount=='"+str(trip_count)+"'").copy()
        df_single_trip = df_single_trip.sort_values(by=['timestamp'])
        max_ts = df_single_trip.timestamp.max()
        min_ts= df_single_trip.timestamp.min()
        
        stay_time =  min_ts - ts_prevs
        trip_time = (max_ts - min_ts).total_seconds()
        
        #if (trip_time < 1800): # ignore if the stay_time is less than 30 minutes
        #    continue
          
        
        lat_min = df_single_trip.iloc[0].lat
        lon_min = df_single_trip.iloc[0].lon
        lat_max = df_single_trip.iloc[len(df_single_trip)-1].lat 
        lon_max = df_single_trip.iloc[len(df_single_trip)-1].lon
        
        avg_lat = (lat_prevs + lat_min)/2
        avg_lon = (lon_prevs + lon_min)/2
        
        lat_prevs = lat_max
        lon_prevs = lon_max
        
        
        arr_trip_summary.append({
            'ap_id': ap_id,
            
            'trip_prevs': trip_count_prevs,
            'trip_count': trip_count,
            'ts_prevs': ts_prevs,
            'ts_min': min_ts,
            #'ts_max': max_ts,#
            #'trip_time': trip_time,#
            'stay_time': stay_time.total_seconds(),
            'avg_lon_with_prvs': avg_lon,
            'avg_lat_with_prvs': avg_lat,
            #'lat_min': lat_min,#
            #'lon_min': lon_min,#
            #'lat_max': lat_max,    #        
            #'lon_max': lon_max #
            })
        trip_count_prevs = trip_count
        ts_prevs = max_ts
    
trip_df =  pd.DataFrame(arr_trip_summary)      
trip_df['date'] = trip_df['ts_min'].dt.date


In [5]:
trip_df.head()

,ap_id,trip_prevs,trip_count,ts_prevs,ts_min,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,date
0,AP521745,2080,2081,2019-04-01 08:24:49,2019-04-01 11:14:20,10171.0,127.748142,26.141869,2019-04-01
1,AP521745,2101,2102,2019-04-01 11:16:50,2019-04-04 18:05:37,283727.0,127.747414,26.140454,2019-04-04
2,AP521745,2123,2124,2019-04-04 18:16:23,2019-04-08 13:51:25,329702.0,127.748549,26.142006,2019-04-08
3,AP521745,2152,2153,2019-04-08 14:05:35,2019-04-13 12:58:27,427972.0,127.749069,26.141213,2019-04-13
4,AP521745,2269,2270,2019-04-13 13:00:56,2019-04-25 14:24:14,1041798.0,127.748285,26.142030,2019-04-25


In [6]:
len(trip_df)

5749

### remove entries with stay_time < 1800 seconds

In [7]:
trip_df1 = trip_df[trip_df['stay_time']>=1800]

In [8]:
len(trip_df1)

5302

### compare with input probes

In [12]:
df=df.sort_values(by=['ap_id','tripcount','timestamp'])
df_temp = df.query("ap_id=='AP521745'")
df_temp[['ap_id','tripcount','timestamp','lat','lon']].head(30)

,ap_id,tripcount,timestamp,lat,lon
49318,AP521745,2080,2019-04-01 08:24:49,26.142524,127.747685
49319,AP521745,2080,2019-04-01 08:25:20,26.143142,127.748158
49320,AP521745,2080,2019-04-01 08:25:38,26.143138,127.748157
49321,AP521745,2080,2019-04-01 08:25:45,26.142988,127.748113
49322,AP521745,2080,2019-04-01 08:26:17,26.141047,127.747722
49323,AP521745,2080,2019-04-01 08:27:08,26.141190,127.748638
49324,AP521745,2080,2019-04-01 08:27:16,26.141196,127.748608
49325,AP521745,2080,2019-04-01 08:27:22,26.141183,127.748630
49326,AP521745,2080,2019-04-01 08:27:40,26.141213,127.748598
49327,AP521745,2080,2019-04-01 08:29:41,26.141213,127.748598
